# TauDEM Raster Processing Notebook

This notebook demonstrates basic GIS raster processing on HydroShare Resources.

In [ ]:
# import some libaries
%matplotlib inline
import os
from hs_utils import hs
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from osgeo import osr, gdal
import subprocess

# get the data directory (this is an environment variable that is provided to you)
data_directory = os.environ['DATA']

### Define some functions that we will use later

1. **Plot_tiff** will provide some simple plotting so that we can see the results of our TauDEM operations
2. **run** provides us an easy way to execute TauDEM tools via the terminal and prints the console output in realtime

In [ ]:
def plot_tiff(tiff, size=(5,5), aspect=1):
    # change the aspect ration to stretch or compress the image
    
    # read the tiff using the gdal library  
    ds = gdal.Open(tiff)
    band = ds.GetRasterBand(1)
    data = band.ReadAsArray()

    # set all negative values (i.e. nodata) to zero so that the map is displayed properly
    data[data<0] = 0

    # create figure to hold plot (figsize=(width, height))
    plt.figure(figsize=size)

    # plot the DEM and display the results
    plt.imshow(data, cmap='gist_earth', aspect=aspect)
    plt.show()

In [ ]:
def run(cmd):
    
    # execute the process
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    
    # Grab stdout line by line as it becomes available.  This will loop until p terminates.
    while p.poll() is None:
        l = p.stdout.readline() # This blocks until it receives a newline.
        print(l.strip().decode('utf-8'))
        
    # When the subprocess terminates there might be unconsumed output 
    # that still needs to be processed.
    print(p.stdout.read().strip().decode('utf-8'))
    

### Create a connection with HydroShare

This step will be replaced with OAuth soon...

In [ ]:
# create a hydroshare intance
hs.getSecureConnection(username=os.environ['HS_USR_NAME'])

### Retrieve a raster resource using its ID

The raster resouce id can be found by navigating to a resource in HydroShare and selecting the ID from the URL.

In [ ]:
# get some resource content. The resource content is returned as a dictionary
content = hs.getResourceContent(os.environ['HS_RES_ID'])

### Perform raster processing

The following operations demonstrate how TauDEM can be used to perform basic raster processes.

In [ ]:
# display the raw dem
raw_dem_path = content['logan.tif']
plot_tiff(raw_dem_path)

In [ ]:
# Fill the DEM Pits

# set the output path for the pitremove operation
fill = os.path.join(data_directory, 'fill.tif')
cmd = 'pitremove -z %s -fel %s' % (raw_dem_path, fill)
print(cmd)
run(cmd)

In [ ]:
# plot the fill result
plot_tiff(fill)

In [ ]:
# Calculate D8 flow direction

# set the output paths for the d8flowdir operation
fdr = os.path.join(data_directory, 'fdr.tif')  # flowdir
sd8 = os.path.join(data_directory, 'sd8.tif')  # slope
cmd = 'd8flowdir -fel %s -sd8 %s -p %s' % (fill, sd8, fdr)
run(cmd)


In [ ]:
# plot the fdr result
plot_tiff(fdr)

In [ ]:
# Calculate D8 contributing area

# set the output paths for the aread8 operation
ad8 = os.path.join(data_directory, 'fdr.tif')  # contributing area
cmd = 'aread8 -p %s -ad8 %s' % (fdr, ad8)
# cmd = 'mpiexec -n 8 aread8 -p %s -ad8 %s' % (fdr, ad8)
run(cmd)

In [ ]:
# plot the contributing area result
plot_tiff(ad8, size=(20,20))

### Save the results back into HydroShare

Using the HydroShare rest api, you can create a new resource in HydroShare from a content file.

In [ ]:
# save this file as a new resource
abstract = 'This is a D8 contributing area raster that was calculated using TauDEM inside a jupyter notebook'
title = 'Logan Contributing Area'
keywords = ('TauDEM', 'Contributing Area', 'JupyterNotebook')
rtype = 'RasterResource'
fpath = ad8
resource_id = h.hs.createResource(rtype, title, resource_file=fpath, keywords=keywords, abstract=abstract)

# make this resource public too
h.hs.setAccessRules(resource_id, public=True)